# **Churn Prediction**

**Churn rate** adalah ukuran jumlah orang atau benda yang masuk atau keluar dari suatu kelompok dalam jangka waktu tertentu. Istilah ini biasanya digunakan dalam konteks customer base sebagai persentase pelanggan yang meninggalkan supplier dalam jangka waktu tertentu.

**Parameter**:
1. **RowNumberRow**: Dari 1 hingga 10000.
2. **CustomerIdUnique**: Id setiap pelanggan bank.
3. **SurnameCustomer**'s: Nama belakang setiap pelanggan bank.
4. **CreditScoreCredit**: Kredit Skor setiap pelanggan.
5. **Geography**: Negara asal pelanggan bank: Jerman, Prancis, Spanyol dan lainnya.
6. **Gender**: Male atau Female.
7. **Age**: Umur pelanggan.
8. **TenureNumber**: Tahun pelanggan menjadi anggota.
9. **BalanceBank**: Saldo pelanggan.
10. **NumOfProductsNumber**: Produk yang digunakan pelanggan.
11. **HasCrCardBinary**: 1: Pelanggan memiliki kartu kredit dan 0: Pelanggan tidak memiliki kartu kredit.
12. **IsActiveMemberBinary**: Aktif atau Pasif pelanggan
13. **EstimatedSalaryEstimated**: Gaji pelanggan

**Target**:

14. **ExitedBinary**: flag 1 if the customer closed account with bank and 0 if the customer is retained

In [3]:
# IMPORT LIBRARY
import keras
import numpy as np
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense

In [7]:
# DOWNLOAD DATA CHURN
!curl -O https://raw.githubusercontent.com/SangsakaWira/Materi-AI-Codedirect/master/Dataset/Churn_Modelling.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  668k  100  668k    0     0  53641      0  0:00:12  0:00:12 --:--:-- 43954     0  57247      0  0:00:11  0:00:09  0:00:02 559520:09  0:00:03 54969


In [5]:
dataset = pd.read_csv("Dataset/Churn_Modelling.csv")

In [6]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [24]:
dataset["Exited"].

2037

# PRE-PROCESSING DATA

Disini kita akan membersihkan data sebelum dimasukkan pada model ANN kita. Ada beberapa data yang perlu dibuang dikarenakan tidak relevan.

In [7]:
# MENGUBAH FORMAT DATA
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [8]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [9]:
# ENCODING CATEGORICAL DATA
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
from sklearn.preprocessing import LabelEncoder
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), [1])],    
    remainder='passthrough'                        
)

# onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

/Users/sangsakawira/miniconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

In [12]:
# SPLIT DATA
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [20]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [21]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 1000, epochs = 100,validation_data=(X_test,y_test))

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
7000/7000 [==============================] - 1s 104us/step - loss: 8.1334 - acc: 0.4849 - val_loss: 3.3307 - val_acc: 0.7930
Epoch 2/100
7000/7000 [==============================] - 0s 40us/step - loss: 3.2598 - acc: 0.7977 - val_loss: 3.3357 - val_acc: 0.7930
Epoch 3/100
7000/7000 [==============================] - 0s 37us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 4/100
7000/7000 [==============================] - 0s 38us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 5/100
7000/7000 [==============================] - 0s 36us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 6/100
7000/7000 [==============================] - 0s 35us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 7/100
7000/7000 [==============================] - 0s 35us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930

7000/7000 [==============================] - 0s 37us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 61/100
7000/7000 [==============================] - 0s 34us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 62/100
7000/7000 [==============================] - 0s 33us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 63/100
7000/7000 [==============================] - 0s 34us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 64/100
7000/7000 [==============================] - 0s 39us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 65/100
7000/7000 [==============================] - 0s 37us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 66/100
7000/7000 [==============================] - 0s 37us/step - loss: 3.2605 - acc: 0.7977 - val_loss: 3.3364 - val_acc: 0.7930
Epoch 67/100
7000/7000 [==============================

In [22]:
x = classifier.predict(X_test)

In [23]:
for i in x:
    print(i)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
